In [1]:
import sys
sys.path.append("../../")  # access to local modules

In [2]:
from functools import partial
from sklearn.datasets import load_sample_images
import tensorflow as tf
import tensorflow_datasets as tfds
import numpy as np
tf.__version__

2025-04-02 14:12:48.536599: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE3 SSE4.1 SSE4.2 AVX AVX2 AVX512F AVX512_VNNI AVX512_BF16 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


'2.17.0'

In [3]:
images = load_sample_images()["images"]
images = tf.keras.layers.CenterCrop(height=70, width=120)(images)
images = tf.keras.layers.Rescaling(scale  = 1 / 255)(images)

2025-04-02 14:12:50.081252: I external/local_xla/xla/stream_executor/rocm/rocm_executor.cc:920] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2025-04-02 14:12:52.199108: I external/local_xla/xla/stream_executor/rocm/rocm_executor.cc:920] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2025-04-02 14:12:52.199152: I external/local_xla/xla/stream_executor/rocm/rocm_executor.cc:920] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2025-04-02 14:12:52.199162: I tensorflow/core/common_runtime/gpu/gpu_device.cc:2397] Ignoring visible gpu device (device: 0, name: AMD Radeon Graphics, pci bus id: 0000:65:00.0) with AMDGPU version : gfx1103. The supported AMDGPU versions are gfx900, gfx906, gfx908, gfx90a, gfx940, gfx941, gfx942, gfx1030, gfx1100,

In [4]:
images.shape

TensorShape([2, 70, 120, 3])

In [5]:
conv_layer = tf.keras.layers.Conv2D(filters=32, kernel_size=7)
fmaps = conv_layer(images)
fmaps.shape

TensorShape([2, 64, 114, 32])

In [6]:
conv_layer = tf.keras.layers.Conv2D(filters=32, kernel_size=7, padding="same")
fmaps = conv_layer(images)
fmaps.shape

TensorShape([2, 70, 120, 32])

In [7]:
kernels, biases = conv_layer.get_weights()
kernels.shape  # [height, width, input channels, output channels]

(7, 7, 3, 32)

In [8]:
biases.shape  # output channels only

(32,)

In [9]:
max_pool = tf.keras.layers.MaxPool2D(pool_size=2)
avg_pool = tf.keras.layers.AvgPool2D(pool_size=2)

In [10]:
class DepthPool(tf.keras.layers.Layer):
    def __init__(self, pool_size=2, **kwargs):
        super().__init__(**kwargs)
        self._pool_size = pool_size

    def call(self, inputs):
        shape = tf.shape(inputs)
        groups = shape[-1] // self.pool_size  # number of channel groups
        new_shape = tf.concat([shape[:-1], [groups, self._pool_size]], axis=0)
        return tf.reduce_max(tf.reshape(inputs, new_shape), axis=-1)

In [11]:
global_avg_pool = tf.keras.layers.GlobalAvgPool2D()  # outputs the means of each feature map
global_avg_pool(images)

<tf.Tensor: shape=(2, 3), dtype=float32, numpy=
array([[0.64338624, 0.5971759 , 0.5824972 ],
       [0.76306933, 0.2601113 , 0.10849128]], dtype=float32)>

In [12]:
DefaultConv2D = partial(tf.keras.layers.Conv2D, kernel_size=3, padding="same", activation="relu", kernel_initializer="he_normal")
model = tf.keras.Sequential([  # in practical nets, use a reshape layer here first
    DefaultConv2D(filters=64, kernel_size=7, input_shape=[28, 28, 1]),
    tf.keras.layers.MaxPool2D(),
    DefaultConv2D(filters=128),
    DefaultConv2D(filters=128),
    tf.keras.layers.MaxPool2D(),
    DefaultConv2D(filters=256),
    DefaultConv2D(filters=256),
    tf.keras.layers.MaxPool2D(),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(units=128, activation="relu", kernel_initializer="he_normal"),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(units=64, activation="relu", kernel_initializer="he_normal"),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(units=10, activation="softmax")
])

/home/dierck/workspace/python/MLTraining/Notebooks/lib/python3.12/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [13]:
mnist = tf.keras.datasets.fashion_mnist.load_data()
(X_train_full, y_train_full), (X_test, y_test) = mnist
X_train_full = np.expand_dims(X_train_full, axis=-1).astype(np.float32) / 255
X_test = np.expand_dims(X_test.astype(np.float32), axis=-1) / 255
X_train, X_valid = X_train_full[:-5000], X_train_full[-5000:]
y_train, y_valid = y_train_full[:-5000], y_train_full[-5000:]

In [14]:
#   takes about 30 minutes to train

# model.compile(loss="sparse_categorical_crossentropy", optimizer="nadam", metrics=["accuracy"])
# history = model.fit(X_train, y_train, epochs=10, validation_data=(X_valid, y_valid))

In [15]:
# score = model.evaluate(X_test, y_test)
# X_new = X_test[:10]  # pretend we have new images
# y_pred = model.predict(X_new)

# ResNet-34 from Scratch

In [16]:
DefaultConv2D = partial(tf.keras.layers.Conv2D, kernel_size=3, strides=1, padding="same", kernel_initializer="he_normal", use_bias=False)

class ResidualUnit(tf.keras.layers.Layer):
    def __init__(self, filters, strides=1, activation="relu", **kwargs):
        super().__init__(**kwargs)
        self._activation = tf.keras.activations.get(activation)
        self._main_layers = [
            DefaultConv2D(filters, strides=strides),
            tf.keras.layers.BatchNormalization(),
            self._activation,
            DefaultConv2D(filters),
            tf.keras.layers.BatchNormalization()
        ]
        self._skip_layers = []
        if strides > 1:
            self._skip_layers = [
                DefaultConv2D(filters, kernel_size=1, strides=strides),
                tf.keras.layers.BatchNormalization()
            ]

    def call(self, inputs):
        Z = inputs
        for layer in self._main_layers:
            Z = layer(Z)
        skip_Z = inputs
        for layer in self._skip_layers:
            skip_Z = layer(skip_Z)

        return self._activation(Z + skip_Z)

In [17]:
model = tf.keras.Sequential([DefaultConv2D(64, kernel_size=7, strides=2, input_shape=[224, 224, 3]),
                             tf.keras.layers.BatchNormalization(),
                             tf.keras.layers.Activation("relu"),
                             tf.keras.layers.MaxPool2D(pool_size=3, strides=2, padding="same")
                            ])
prev_filters = 64
for filters in [64] * 3 + [128] * 4 + [256] * 6 + [512] * 3:
    strides = 1 if filters == prev_filters else 2
    model.add(ResidualUnit(filters, strides=strides))
    prev_filters = filters

model.add(tf.keras.layers.GlobalAvgPool2D())
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(10, activation="softmax"))

In [18]:
K = tf.keras.backend
images = K.constant(load_sample_images()["images"])
print(len(images))
images_resized = tf.keras.layers.Resizing(height=224, width=224, crop_to_aspect_ratio=True)(images)

2


In [19]:
inputs = tf.keras.applications.resnet50.preprocess_input(images_resized)

In [20]:
model = tf.keras.applications.ResNet50(weights="imagenet")

In [21]:
Y_proba = model.predict(inputs)
Y_proba.shape  # (number of images, classes)

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 790ms/step


(2, 1000)

In [22]:
len(images)

2

In [23]:
top_K = tf.keras.applications.resnet50.decode_predictions(Y_proba, top=3)
for image_index in range(len(images)):
    print(f"Image #{image_index}")
    for class_id, name, y_proba in top_K[image_index]:
        print(f"\t{class_id} - {name:12s} {y_proba:-2%}")

Image #0
	n03877845 - palace       54.693586%
	n03781244 - monastery    24.714711%
	n02825657 - bell_cote    18.550421%
Image #1
	n04522168 - vase         32.669640%
	n11939491 - daisy        17.817330%
	n03530642 - honeycomb    12.041561%


# Transfer Learning

In [24]:
dataset, info = tfds.load("tf_flowers", as_supervised=True, with_info=True)

In [25]:
datset_size = info.splits["train"].num_examples
class_names = info.features["label"].names
n_classes = info.features["label"].num_classes

In [26]:
test_set_raw, valid_set_raw, train_set_raw = tfds.load("tf_flowers", split=["train[:10%]", "train[10%:25%]", "train[25%:]"], as_supervised=True)

In [27]:
batch_size = 32
preprocess = tf.keras.Sequential([
    tf.keras.layers.Resizing(height=224, width=224, crop_to_aspect_ratio=True),
    tf.keras.layers.Lambda(tf.keras.applications.xception.preprocess_input)
])

In [28]:
train_set = train_set_raw.map(lambda X, y: (preprocess(X), y))
train_set = train_set.shuffle(1000, seed=42).batch(batch_size).prefetch(1)
valid_set = valid_set_raw.map(lambda X, y: (preprocess(X), y)).batch(batch_size)
test_set = test_set_raw.map(lambda X, y: (preprocess(X), y)).batch(batch_size)

In [29]:
data_augmentation = tf.keras.Sequential([
    tf.keras.layers.RandomFlip(mode="horizontal", seed=42),
    tf.keras.layers.RandomRotation(factor=0.05, seed=42),
    tf.keras.layers.RandomContrast(factor=0.2, seed=42)
])

In [30]:
base_model = tf.keras.applications.xception.Xception(weights="imagenet", include_top=False)
avg = tf.keras.layers.GlobalAveragePooling2D()(base_model.output)
output = tf.keras.layers.Dense(n_classes, activation="softmax")(avg)
model = tf.keras.Model(inputs=base_model.input, outputs=output)

In [31]:
for layer in base_model.layers:
    layer.trainable = False

In [32]:
optimizer = tf.keras.optimizers.SGD(learning_rate=0.1, momentum=0.9)
model.compile(loss="sparse_categorical_crossentropy", optimizer=optimizer, metrics=["accuracy"])
# history = model.fit(train_set, validation_data=valid_set, epochs=3)   # takes about 8 minutes

In [33]:
for layer in base_model.layers[56:]:  # unfreeze residual unit 7 out of 14 and above
    layer.trainable = True

optimizer = tf.keras.optimizers.SGD(learning_rate=0.01, momentum=0.9)
model.compile(loss="sparse_categorical_crossentropy", optimizer=optimizer, metrics=["accuracy"])
# history = model.fit(train_set, validation_data=valid_set, epochs=10)  #  takes about 40 minutes...

# Localization

In [35]:
base_model = tf.keras.applications.xception.Xception(weights="imagenet", include_top=False)
avg = tf.keras.layers.GlobalAveragePooling2D()(base_model.output)
class_output = tf.keras.layers.Dense(n_classes, activation="softmax")(avg)
loc_output = tf.keras.layers.Dense(4)(avg)
model = tf.keras.Model(inputs=base_model.input, outputs = [class_output, loc_output])
model.compile(loss=["sparse_categorical_crossentropy", "mse"], loss_weights = [0.8, 0.2], optimizer=optimizer, metrics=["accuracy"])
# missing labels, and training would take very long time...